In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from pygame import mixer
from collections import deque
import time

In [ ]:
frame_counter = 0
prev_time = time.time()

Alarm setup

In [ ]:
mixer.init()
try:
    sound = mixer.Sound('alarm.wav')
except:
    print("Alarm file not found. Please ensure 'alarm.wav' is in the same directory.")
    sound = None

Classifiers for face and eyes

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml')
leye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_lefteye_2splits.xml')
reye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_righteye_2splits.xml')

Load trained CNN model

In [ ]:
model = tf.keras.models.load_model('bestModel.keras')

Preprocess for CNN

In [ ]:
def preprocess_eye(img):
    img = cv2.resize(img, (64, 64)) / 255.0
    # No more converting grayscale to 3 channels
    img = np.expand_dims(img, axis=-1)  # Add channel dimension for grayscale
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

Initialize

In [ ]:
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
score = 0  # Start at zero - only used for alarm, not status
thicc = 2
CONSEC_FRAMES = 5 # Adjust this - still relevant for ALARM
METHOD_USED = "None" # Consistent variable naming
eye_status_history = deque(maxlen=3) # initialize history

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        print("Camera error")
        break

        # FPS LOGIC
    frame_counter += 1
    if frame_counter == 10:
        curr_time = time.time()
        elapsed_time = curr_time - prev_time
        fps = frame_counter / elapsed_time
        print(f"FPS: {fps:.2f}")
        frame_counter = 0
        prev_time = curr_time
    height, width = frame.shape[:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Face detection using Haar Cascade
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)
    left_eye_detected = right_eye_detected = False
    lpred = rpred = 1 #Initialize to open
    if len(faces) > 0:
        for (fx, fy, fw, fh) in faces:
            # Draw rectangle around the face
            cv2.rectangle(frame, (fx, fy), (fx + fw, fy + fh), (255, 0, 0), 2)
            face_roi = gray[fy:fy+fh, fx:fx+fw]

            # Define Eye Regions
            eye_region_height = int(fh * 0.3)  # 30% of face height
            eye_region_width = int(fw * 0.4)   # 40% of face width
            left_eye_x = int(fw * 0.1)        # Left side, 10% from the edge
            left_eye_y = int(fh * 0.2)        # Top, 20% from the top
            right_eye_x = int(fw * 0.5)       # Right side, start at 50%
            right_eye_y = int(fh * 0.2)        # Top, 20% from the top

            # Extract Eye ROIs.  Handle edge cases where ROI might be out of bounds
            try:
                left_eye_roi = face_roi[left_eye_y:left_eye_y + eye_region_height, left_eye_x:left_eye_x + eye_region_width]
                right_eye_roi = face_roi[right_eye_y:right_eye_y + eye_region_height, right_eye_x:right_eye_x + eye_region_width]
            except IndexError:
                print("Eye ROI out of bounds. Skipping this face.")
                continue # Skip to the next face

            # Detect eyes within the defined regions
            left_eyes = leye_cascade.detectMultiScale(left_eye_roi, scaleFactor=1.1, minNeighbors=5)
            right_eyes = reye_cascade.detectMultiScale(right_eye_roi, scaleFactor=1.1, minNeighbors=5)
            try:
                lprob = 0.0 # initialize
                rprob = 0.0 # initialize
                if len(left_eyes) > 0:
                    (lx, ly, lw, lh) = left_eyes[0]
                    # Adjust coordinates to be relative to the *face_roi*, not the *left_eye_roi*
                    lx_adj = left_eye_x + lx
                    ly_adj = left_eye_y + ly
                    # FORCE TO GRAYSCALE
                    l_eye = face_roi[ly_adj:ly_adj+lh, lx_adj:lx_adj+lw]
                    l_input = preprocess_eye(l_eye)
                    lprob = model.predict(l_input, verbose=0)[0][0]  # Get the probability
                    lpred = 1 if lprob > 0.5 else 0
                    left_eye_detected = True
                    cv2.rectangle(frame, (fx + lx_adj, fy + ly_adj), (fx + lx_adj + lw, fy + ly_adj + lh), (0, 255, 0), 2)
                if len(right_eyes) > 0:
                    (rx, ry, rw, rh) = right_eyes[0]
                    # Adjust coordinates to be relative to the *face_roi*, not the *right_eye_roi*
                    rx_adj = right_eye_x + rx
                    ry_adj = right_eye_y + ry
                    # FORCE TO GRAYSCALE
                    r_eye =  face_roi[ry_adj:ry_adj+rh, rx_adj:rx_adj+rw]
                    r_input = preprocess_eye(r_eye)
                    rprob = model.predict(r_input, verbose=0)[0][0]  # Get the probability
                    rpred = 1 if rprob > 0.5 else 0
                    right_eye_detected = True
                    cv2.rectangle(frame, (fx + rx_adj, fy + ry_adj), (fx + rx_adj + rw, fy + ry_adj + rh), (0, 0, 255), 2)
            except Exception as e:
                print(f"Error during eye processing: {e}")
                pass
            if left_eye_detected or right_eye_detected:
                METHOD_USED = "Haar + CNN"

    # Determine STATUS based ONLY on current frame's model predictions
    CNN_closed = (lpred == 0 and rpred == 0)  # Both eyes predicted closed

    # ADD HISTORY TO SMOOTH PREDICTIONS
    if CNN_closed:
        eye_status_history.append(1)
    else:
        eye_status_history.append(0)

    # Decide status and calculate the score
    if sum(eye_status_history) >= 2: # This line is same to the provided code
        score += 1
        status = "Closed" # Status determined by HISTORY
    else:
        score -= 1
        status = "Open"  # Status determined by HISTORY
    score = max(0, score) # cap the score

In [ ]:
    int_score = int(score)

    # Print the status and score to the console
    print(f"Frame Status: {status}, Score: {int_score}")

    # Display information on the frame
    cv2.putText(frame, f"Status: {status}", (10, height - 50), font, 1, (255, 255, 255), 1)
    cv2.putText(frame, f"Score: {int_score}", (10, height - 30), font, 1, (255, 255, 255), 1)  # Display as int
    cv2.putText(frame, f"Method: {METHOD_USED}", (10, height - 10), font, 1, (0, 255, 255), 1)

    # Alarm logic - BASED on SCORE, NOT STATUS
    if score > CONSEC_FRAMES:
        if sound and not mixer.get_busy():
            sound.play()
        cv2.rectangle(frame, (0, 0), (width, height), (0, 0, 255), thicc)
        thicc = thicc + 2 if thicc < 16 else thicc - 2
        thicc = max(2, thicc)
    else:
        if sound:
            sound.stop()
    cv2.imshow('Drowsiness Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
cap.release()
cv2.destroyAllWindows()
if sound:
    mixer.quit()